# Markov decision process(MDP)

에이전트에게 완전한 문제를 제시하는 환경을 마르코프 결정 과정(Markov decision process, MDP)라고 한다. <br>
이러한 환경은 주어진 환경에서의 보상과 상태 전이를 제공하는 데 그치지 않고, <br>
환경의 상태와 에이전트가 자신의 상태에서 취하는 액션에 의해 좌우, 시간에 의해 보상이 달라질 수 있음 <br>
에이전트는 어느 시점에서 S(상태의 집합)에서 s(상태)를 경험하게 된다. 또한 A가 존재하며 이는 에이전트가 취할 수 있는 액션의 집합으로 표현된다

In [24]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt

In [25]:
# 오픈AI에서 특정 환경을 불러오는 코드
env = gym.make("CartPole-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [26]:
# CartPole 환경의 액션과 관측값의 종류
env.action_space, env.observation_space

(Discrete(2), Box(4,))

In [30]:
# 정책 기반 에이전트
gamma = 0.99

def discount_rewards(r):
    # 보상의 1D 실수 배열을 취해서 할인된 보상을 계산
    discounted_r = np.zeros_like(r)
    running_add = 0
    
    #for t in range(r.size-1, -1, -1):
    for t in reversed(range(0, r.size)):
        running_add = running_add*gamma+r[t]
        discounted_r[t] = running_add
    
    return discounted_r

class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        # 네트워크의 피드포워드 부분, 에이전트는 상태를 받아서 액션을 출력
        self.state_in = tf.placeholder(shape=[None, s_size], dtype=tf.float32)
        # 히든레이어 설정
        hidden = slim.fully_connected(inputs=self.state_in, num_outputs=h_size, 
                                      activation_fn=tf.nn.relu, biases_initializer=None)
        # 출력레이어 설정
        self.output = slim.fully_connected(inputs=hidden, num_outputs=a_size, 
                                           activation_fn=tf.nn.softmax, biases_initializer=None)
        # 출력레이어에서 값이 가장 큰 값의 인덱스 선택
        self.chosen_action = tf.argmax(self.output, axis=1)
        
        # 학습 과정 구현, 비용을 계산하기 위해 보상과 액션을 네트워크에 피드하고, 이를 통해 네트워크 업데이트
        self.reward_holder = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype=tf.int32)
        
        self.indexes = tf.range(start=0, limit=tf.shape(input=self.output)[0])*tf.shape(input=self.output)[1]+self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)
        
        # 평균값 줄이기
        self.loss = -tf.reduce_mean(input_tensor=tf.log(self.responsible_outputs)*self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx, ele in enumerate(tvars):
            placeholder = tf.placeholder(dtype=tf.float32, name=str(idx)+"_holder")
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(ys=self.loss, xs=tvars)
        
        optimizer =tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))

In [31]:
# 에이전트 학습

# 텐서플로우 그래프 리셋
tf.reset_default_graph()

# 에이전트 로드(상태는 네 가지, 액션은 두 가지)
myAgent = agent(lr=1e-2, s_size=4, a_size=2, h_size=8)

# 에이전트를 학습시킬 총 에피소드 수
total_episodes = 5000
max_ep = 999
update_frequency = 5

init = tf.global_variables_initializer()

# 텐서플로우 그래프 런칭
with tf.Session() as sess:
    sess.run(init)
    total_reward = []
    total_length = []
    
    gradBuffer = sess.run(tf.trainable_variables())
    for idx, grad in enumerate(gradBuffer):
        gradBuffer[idx] = grad*0
    
    for i in range(total_episodes):
        s = env.reset()
        running_reward = 0
        ep_history = []
        for j in range(max_ep):
            # 네트워크 출력에서 확률적으로 액션을 선택
            a_dist = sess.run(myAgent.output, feed_dict={myAgent.state_in:[s]})
            a = np.random.choice(a_dist[0], p=a_dist[0])
            a = np.argmax(a_dist == a)
            
            # 주어진 밴딧에 대해 액션을 취한 보상
            s1, r, d, _ = env.step(a) #observation, reward, done, info가 리턴
            ep_history.append([s, a, r, s1])
            s = s1
            running_reward += r
            if d == True:
                # 네트워크 업데이트
                ep_history = np.array(ep_history)
                ep_history[:, 2] = discount_rewards(ep_history[:, 2]) # 보상을 디스카운트
                feed_dict = {myAgent.reward_holder:ep_history[:, 2], myAgent.action_holder:ep_history[:, 1], myAgent.state_in:np.vstack(ep_history[:, 0])}
                grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
                for idx, grad in enumerate(grads):
                    gradBuffer[idx] += grad
            
                if i % update_frequency == 0 and i !=0:
                    feed_dict = dict(zip(myAgent.gradient_holders, gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                    for idx, grad in enumerate(gradBuffer):
                        gradBuffer[idx] = grad*0
            
                total_reward.append(running_reward)
                total_length.append(j)
                break
        
        # 보상의 총계 업데이트
        if i % 100 == 0:
            print(np.mean(total_reward[-100:]))
        

C:\Users\a\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


13.0
30.6
34.87
48.35
60.27
69.47
121.23
154.68
160.27
179.84
193.28
194.93
197.82
194.56
193.73
195.89
191.9
196.3
198.55
195.89
197.2
196.15
195.08
191.81
186.04
193.54
197.36
197.87
198.88
191.59
183.21
191.05
194.31
184.74
192.39
199.84
197.21
192.08
187.27
183.42
193.05
196.6
198.39
200.0
199.97
199.45
198.44
198.42
199.62
199.83
